In [1]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt

from astropy.table import Table
from astropy import units as u

from astropy.nddata import CCDData, StdDevUncertainty
from specutils import Spectrum1D
import ccdproc
from astropy.stats import mad_std
from ccdproc import trim_image, Combiner


In [2]:
from astropy.io import fits
from time import gmtime, strftime

def combine_fits(files_data: list[np], operation, outfile: str) -> None:
    img_array = []
    hdu_array = []
    nb_images = 0
   
    #median_array = np.median(img_array, axis=0)
    median_array = operation(img_array, axis=0)
    out_file = fits.PrimaryHDU(median_array, hdu_array[0])
    out_hdr = out_file.header
    out_hdr['DATE'] = strftime("%Y-%m-%d %H:%M:%S", gmtime())
    out_hdr['history'] = 'Median of %d images by numpy.median' %(nb_images,)
    out_hdr['history'] = 'First image '+  files[0]
    out_hdr['history'] = 'Last image  '+  files[nb_images - 1]
    out_file.writeto(outfile, overwrite = True)


In [19]:
import warnings
import fnmatch
import os
import ccdproc
from astropy.utils.exceptions import AstropyWarning
from time import gmtime, strftime
#warnings.simplefilter('ignore', category=AstropyWarning)

dir = 'e:/Astro/Captures/20231008_Void/'
bias_files = [dir + f for f in fnmatch.filter(os.listdir(dir), 'Bias*')]
dark_files = [dir + f for f in fnmatch.filter(os.listdir(dir), 'dark*')]
flat_files = [dir + f for f in fnmatch.filter(os.listdir(dir), 'flat*')]
sci_files  = [dir + f for f in fnmatch.filter(os.listdir(dir), 'deneb*')]

print ('bias combine...')
bias_datas = [CCDData.read(f, unit = u.adu) for f in bias_files]
bias_median = ccdproc.combine(bias_datas,method='average',
    sigma_clip = True, sigma_clip_low_thresh = 5, sigma_clip_high_thresh = 5,
    sigma_clip_func = np.ma.median, signma_clip_dev_func = mad_std, 
    dtype = np.int16, mem_limit = 1e9)

out_file = fits.PrimaryHDU(data=bias_median.data.astype(np.uint32), header=bias_median.header)
out_hdr = out_file.header
out_hdr['DATE'] = strftime("%Y-%m-%d %H:%M:%S", gmtime())
out_hdr['HISTORY'] = 'CCDProc.combine - sygma-clip'
out_file.writeto(dir + 'masterbias.fits', overwrite = True)

#bias_median.write(dir + 'masterbias.fits', overwrite=True)
bias_datas = None
bias_median = None

"""
combined_dark = ccdp.combine(calibrated_darks,
                                 method='average',
                                 sigma_clip=True, sigma_clip_low_thresh=5, sigma_clip_high_thresh=5,
                                 sigma_clip_func=np.ma.median, signma_clip_dev_func=mad_std,
                                 mem_limit=350e6
                                )

    combined_dark.meta['combined'] = True

    dark_file_name = 'combined_dark_{:6.3f}.fit'.format(exp_time)
    combined_dark.write(calibrated_path / dark_file_name)
    
print ('dark combine...')
dark_datas = [ccdproc.subtract_bias(CCDData.read(f, unit = u.adu), bias_median) for f in dark_files]
print('memory = ' + dark_datas.nbytes)
#dark_median = np.median(dark_datas, axis=0)
#dark_median = CCDData(np.median(dark_datas, axis=0), unit="adu")
#dark_median.write(dir + 'masterdark.fits', overwrite=True)
#dark_datas = None

dark_datas = None
ccd = ccdproc.subtract_bias(ccd, master_bias_red)
print ('flat combine...')
flat_datas = [(fits.open(f)[0].data.astype('float32') - bias_median) for f in flat_files]
dark_sub = ccdproc.subtract_bias(dark, masterbias)
flat_median = CCDData(np.median(flat_datas, axis=0), unit="adu")
flat_median.write(dir + 'masterflat.fits', overwrite=True)
flat_datas = None

print ('science combine...')
sci_datas = [((fits.open(f)[0].data.astype('float32') - dark_median - bias_median) / flat_median) for f in sci_files]
sci_add = CCDData(np.sum(sci_datas, axis=0, unit="adu"))
ccd.write(dir + 'mastersci.fits', overwrite=True)

sci_datas = None
bias_median = None
dark_median = None
flat_median = None
sci_add = None
"""

bias combine...


INFO: splitting each image into 4 chunks to limit memory usage to 1000000000.0 bytes. [ccdproc.combiner]


'\ncombined_dark = ccdp.combine(calibrated_darks,\n                                 method=\'average\',\n                                 sigma_clip=True, sigma_clip_low_thresh=5, sigma_clip_high_thresh=5,\n                                 sigma_clip_func=np.ma.median, signma_clip_dev_func=mad_std,\n                                 mem_limit=350e6\n                                )\n\n    combined_dark.meta[\'combined\'] = True\n\n    dark_file_name = \'combined_dark_{:6.3f}.fit\'.format(exp_time)\n    combined_dark.write(calibrated_path / dark_file_name)\n    \nprint (\'dark combine...\')\ndark_datas = [ccdproc.subtract_bias(CCDData.read(f, unit = u.adu), bias_median) for f in dark_files]\nprint(\'memory = \' + dark_datas.nbytes)\n#dark_median = np.median(dark_datas, axis=0)\n#dark_median = CCDData(np.median(dark_datas, axis=0), unit="adu")\n#dark_median.write(dir + \'masterdark.fits\', overwrite=True)\n#dark_datas = None\n\ndark_datas = None\nccd = ccdproc.subtract_bias(ccd, master_b

In [14]:
bias_datas = None
bias_median = None

In [ ]:
"""
inhdr = inlist[0].header
darkhdr = darklist[0].header 
inexp = float(inhdr['EXPTIME'])
darkexp = float(darkhdr['EXPTIME'])
darkscale = inexp/darkexp


# Create a dark-subtracted image

outimage = (inimage -inbias) - darkscale * (indark -inbias)

combine_fits(bias_files, np.median, dir + 'masterbias.fit')

darks_list = []
masterbias = CCDData.read(dir + 'masterbias.fit', unit = u.adu)
for dark in darkfiles:
    dark = CCDData.read(dark, unit = u.adu)
    dark_sub = ccdp.subtract_bias(dark, masterbias)
    darks_list.append(dark_sub)

dark_sub_file = ccdproc.combine(blue_flat_list, method='median')
dark_sub_file.write('master_flat_blue.fits', clobber=True)



combine_fits(biasfiles, np.median, dir + 'masterdark.fit')
"""

In [ ]:
plt.figure(figsize=(10,6))
plt.imshow(outimage, origin='lower', aspect='auto', cmap=plt.cm.inferno)
plt.title('outimage')
plt.clim(np.percentile(outimage, (5, 98)))
cb = plt.colorbar()

In [ ]:
import fnmatch
import os
dir = 'e:/Astro/Captures/20231008_Void/'
biasfiles = [dir + f for f in fnmatch.filter(os.listdir(dir), 'Bias*')]
combine_median(biasfiles, dir + 'masterbias.fit')
scifile = 'deneb-200s-1.fit'
masterbias = CCDData.read(dir + 'masterbias.fit', unit = u.adu)
ccd_bias = ccdp.subtract_bias(ccd, masterbias)

#masterdark = CCDData.read(dir + 'masterdark.fit', unit = u.adu)
#masterflat = CCDData.read(dir + 'masterflat.fit', unit = u.adu)
ccd_dark = ccdp.subtract_dark(ccd_bias, masterdark, exposure_unit=u.second, scale = True, dark_exposure = 300.0*u.second, data_exposure = 200.0*u.second)
ccd = ccdp.flat_correct(ccd_dark, masterflat, min_value=0.9)
ccd = CCDData.read(dir + scifile, unit = u.adu)
plt.figure(figsize=(10,6))
plt.imshow(ccd, origin='lower', aspect='auto', cmap=plt.cm.gray)
plt.title('outimage')
#plt.clim(np.percentile(ccd, (5, 98)))
cb = plt.colorbar()

In [ ]:
from specreduce import tracing, background, extract
sci_tr = tracing.FitTrace(ccd,  peak_method='max')   #FitTrace(image, peak_method='gaussian', guess=trace_pos)
bg = background.Background.two_sided(ccd, sci_tr, separation=50, width=20) 
extract = extract.BoxcarExtract(ccd - bg, sci_tr, width = 10)
spectrum = extract()


In [ ]:
extract.width


In [ ]:
plt.figure(figsize=(10,6))
plt.imshow(ccd, origin='lower', aspect='auto', cmap=plt.cm.gray)
plt.imshow(bg.bkg_wimage, origin='lower', aspect='auto', cmap=plt.cm.gray, alpha=0.5)
plt.plot(sci_tr.trace , color='r')
plt.plot(sci_tr.trace+extract.width , color='g', linestyle='dashed')
plt.plot(sci_tr.trace-extract.width , color='g', linestyle='dashed')

cb = plt.colorbar()



In [ ]:
plt.figure(figsize=(10,6))
#plt.imshow(bg.sub_image(ccd).data, origin='lower', aspect='auto', cmap=plt.cm.gray)
#plt.imshow(bg.sub_image(ccd).data, origin='lower', aspect='auto', cmap=plt.cm.gray)
plt.plot(spectrum.flux, color='r')
#cb = plt.colorbar()

